# Test Model using Mediapipe

In [4]:
# Libraries
import numpy as np
import joblib
import cv2
import mediapipe as mp
from collections import deque, Counter

import warnings
warnings.filterwarnings("ignore")

# Load the trained model and encoder
model = joblib.load('xgb_model.pkl')
encoder = joblib.load('encoder.pkl')

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)

# Initialize deque for output stabilization
window_size = 15
gesture_history = deque(maxlen=window_size)

# Start video capture
cap = cv2.VideoCapture(0)

# Get frame width and height
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Initialize VideoWriter for recording (MP4 format)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, 20.0, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)  # Flip horizontally
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:

            # Extract landmarks
            landmarks = np.array([[lm.x, lm.y, lm.z] for lm in hand_landmarks.landmark])
            
            if landmarks.shape == (21, 3):  # Ensure hand is detected
                # Normalize hand coordinates
                x_scaler = landmarks[12, 0] - landmarks[0, 0]
                y_scaler = landmarks[12, 1] - landmarks[0, 1]
                landmarks[:, 0] = (landmarks[:, 0] - landmarks[0, 0]) / (x_scaler)  
                landmarks[:, 1] = (landmarks[:, 1] - landmarks[0, 1]) / (y_scaler) 
            
                # Predict gesture
                prediction = model.predict([landmarks.flatten()])[0]
                label = encoder.inverse_transform([prediction])[0]                
                gesture_history.append(label)
                gesture = Counter(gesture_history).most_common(1)[0][0]
            
                # Draw landmarks
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                
                # Display gesture label
                cv2.putText(frame, f"Gesture: {gesture}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    
    else:
        cv2.putText(frame, "No Hand Detected", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Write frame to MP4 file
    out.write(frame)

    # Show the frame
    cv2.imshow('Hand Gesture Recognition', frame)
    
    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()  # Release VideoWriter
cv2.destroyAllWindows()
